In [261]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

In [262]:
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True) #no scientific notation
from scipy.stats import chi2_contingency

In [263]:
df = pd.read_csv(r"Geo Modified Dataset - 80 cells - width==0.1 .csv", index_col=(0,1))

# Add Interaction Terms

In [264]:
race_set = {'ASIAN', 'BLACK', 'HISPANIC', 'NATIVE AMERICAN', 'OTHER', 'WHITE'}

no_NA_race_set = {'ASIAN', 'BLACK', 'HISPANIC', 'OTHER', 'WHITE'}

for race1 in no_NA_race_set:
    
    for race2 in no_NA_race_set:
        df[f"INTERACTION: D_{race1} X {race2} Racial Composition"] = df[f'{race1} - (D_Race)'] * df[f'GEO: {race2} Racial Composition']

        df[f"INTERACTION: D_{race1} X {race2} Percent of Charges that were CHANGED"] = df[f'{race1} - (D_Race)'] * df[f'GEO: {race2} Percent of Charges that were CHANGED']

        df[f"INTERACTION: D_{race1} X {race2} Average Speed NOT in 9,14 MPH"] = df[f'{race1} - (D_Race)'] * df[f'GEO: {race2} Average Speed NOT in 9,14 MPH']

        df[f"INTERACTION: D_{race1} X D_Male"] = df[f'{race1} - (D_Race)'] * df["Male"]

# Chi-Squared Test for Independence

In [265]:
# race_set = {'ASIAN', 'BLACK', 'HISPANIC', 'NATIVE AMERICAN', 'OTHER', 'WHITE'}
speeding_bool_set = {"Speed Altered", 'Speed NOT Altered'}

In [266]:
altered = []
not_altered = []


for x in df['Speed Over Posted Limit']:
    if x==9:
        altered.append(1)
        not_altered.append(0)
    elif 10 <= x <= 14:
        altered.append(0)
        not_altered.append(1)
    else:
        altered.append(np.nan)
        not_altered.append(np.nan)

df['Speed Altered'] = altered
df['Speed NOT Altered'] = not_altered

In [267]:
contingency_table = pd.DataFrame({x:[0 for race in sorted(list(race_set))] for x in speeding_bool_set}, index=sorted(list(race_set)))

In [268]:
for col in contingency_table:
    for ind in contingency_table.index:
        temp_col = []
        for x in zip(df[col], df[f"{ind} - (D_Race)"]):
            temp_col.append(x[0]==x[1]==1)
            
        contingency_table[col].loc[ind] = sum(temp_col)

        
contingency_table

,Speed Altered,Speed NOT Altered
ASIAN,3626,236
BLACK,10670,1049
HISPANIC,7039,726
NATIVE AMERICAN,36,10
OTHER,3179,267
WHITE,24843,1479


## Chi2 Test

Returns (in order):

- The test statistic.

- The p-value of the test

- Degrees of freedom

- The expected frequencies, based on the marginal sums of the table.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html

In [269]:
chi2_result = chi2_contingency(contingency_table)

print(f"\n Test statistic == {chi2_result[0]}")
print(f"\n P-Value == {chi2_result[1]}")
print(f"\n Degrees of Freedom == {chi2_result[2]}")
print(f"\n Expected Frequencies == \n {chi2_result[3]}")

print(f"\n Difference between Actual and Expected Frequencies (Actual - Expected) == \n\n {contingency_table-chi2_result[3]}")


 Test statistic == 231.28605043787022

 P-Value == 5.66938794054964e-48

 Degrees of Freedom == 5

 Expected Frequencies == 
 [[ 3588.33269375   273.66730625]
 [10888.57349511   830.42650489]
 [ 7214.76006396   550.23993604]
 [   42.7403687      3.2596313 ]
 [ 3201.81109857   244.18890143]
 [24456.78227991  1865.21772009]]

 Difference between Actual and Expected Frequencies (Actual - Expected) == 

                  Speed Altered  Speed NOT Altered
ASIAN                37.667306         -37.667306
BLACK              -218.573495         218.573495
HISPANIC           -175.760064         175.760064
NATIVE AMERICAN      -6.740369           6.740369
OTHER               -22.811099          22.811099
WHITE               386.217720        -386.217720


# Regression

Maybe do this in R - Python implementation is not great (scipy doesn't have a regression summary / p-value; statsmodel doesn't make sense).

## Set up

In [270]:
regression_df = df[(9<=df['Speed Over Posted Limit']) & (df['Speed Over Posted Limit']<=14)] # all observations where 9 <= speed <= 14 

#replace np.nan with 0
for x in list(zip(regression_df.isnull().sum(), regression_df.columns)):
    if x[0]!=0:
        regression_df[x[1]] = regression_df[x[1]].fillna(value=0)
    
y = regression_df['Speed Altered']

C:\Users\mikha\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Deletions

#### Delete non-boolean or non-float columns

In [271]:
for col in regression_df.columns:
    if regression_df[col].dtype not in ('bool', 'float64', 'int64'):
        if regression_df[col].dtype=='object':
            del regression_df[col]    

#### Delete Multicollinearity-Causing Columns

In [272]:
multicollinear_cols = ['GEO: OTHER Racial Composition',
'OTHER - (D_Race)',
'Headquarters and Special Operations - (D_SubAgency)',
'ESERO - (D_Violation Type)',
'Number of writeups'
                      ]

for col in multicollinear_cols:
    try:
        del regression_df[col]
    except Exception as e:
        print(f"Failed to delete {col}, exception: {e}")

#### Other Deletions

In [273]:
misc_del_list = set(
['Speed Altered',
'Speed NOT Altered',
    
'Citation - (D_Violation Type)',
'Warning - (D_Violation Type)',
            
'Citation - (D_Search Outcome)',
'Warning - (D_Search Outcome)',            
])

for col in regression_df:
    if 'D_Search Outcome' in col:
        misc_del_list.add(col)

for col in misc_del_list:
    
    try:
        
        del regression_df[col]
    

    except Exception as e:
        
        print(f"failed to delete {col}, exception: {e}")

X = regression_df

In [274]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1) #keep this random_state for reproducibility

### Confirm that there's an even proportion of True, False in training and testing sets

In [275]:
print(f"mean(y_test)-mean(y_train) == {(abs(np.mean(y_test)-np.mean(y_train)))}")

print(f'Discrepancy in raw count == { (abs(np.mean(y_test)-np.mean(y_train)))*len(regression_df)}')

mean(y_test)-mean(y_train) == 0.00018811136192620204
Discrepancy in raw count == 9.9999999999969


## Running the Regression

Questions:

- Do I need to normalize the data before L1?
- Should we add interaction terms for the geo vars?

Thoughts:

- Results with/without L1 are very different
  * All Geo vars have the same coefficients in normal logit

### With an Intercept

In [276]:
# pd.options.display.width = 0

In [277]:
l1_model = LogisticRegression(penalty='l1', solver='liblinear')
l1_model.fit(X, y)
l1_coefficients = l1_model.coef_.tolist()[0]
zipped_l1_coefs = list(zip([['Intercept']+list(regression_df.columns)][0], [l1_model.intercept_.tolist()[0]]+[round(x,3) for x in l1_coefficients]))



log_model = LogisticRegression(max_iter=8000) #default max_iter==100
log_model.fit(X, y)
log_coefficients = log_model.coef_.tolist()[0]
zipped_log_coefs = list(zip(['Intercept']+[list(regression_df.columns)][0], [log_model.intercept_.tolist()[0]]+[round(x,3) for x in log_coefficients]))

# OLS_model = LinearRegression()
# OLS_model.fit(X, y)
# OLS_coefficients = log_model.coef_.tolist()[0]
# zipped_OLS_coefs = list(zip(['Intercept']+[list(regression_df.columns)][0], [OLS_model.intercept_.tolist()[0]]+[round(x,3) for x in OLS_coefficients]))


results_df = pd.DataFrame({'Variable': [x[0] for x in zipped_l1_coefs],
                           'L1 Coefficient': [x[1] for x in zipped_l1_coefs],
                           'Normal Logit Coefficient': [x[1] for x in zipped_log_coefs],
#                            'OLS Coefficient': [x[1] for x in zipped_OLS_coefs]
                          })

results_df = results_df.sort_values(by=['L1 Coefficient'], ascending=False)

C:\Users\mikha\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [278]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 0):
    print(results_df)

                                              Variable  L1 Coefficient  Normal Logit Coefficient
0                                            Intercept       23.451998                 22.628433
21                                    WHITE - (D_Race)        4.403000                  7.991000
18                                    BLACK - (D_Race)        4.090000                  5.584000
17                                    ASIAN - (D_Race)        4.084000                  2.497000
19                                 HISPANIC - (D_Race)        4.013000                  3.834000
20                          NATIVE AMERICAN - (D_Race)        2.888000                  0.052000
112                      INTERACTION: D_OTHER X D_Male        1.609000                  0.851000
22             1st District, Rockville - (D_SubAgency)        1.156000                  3.842000
25               4th District, Wheaton - (D_SubAgency)        1.129000                  3.664000
8                             

#### Prediction Accuracy

~~**NOTE: THIS WAS NOT SCORING 100% UNTIL ADDING RACExGEO, RACExMALE INTERACTION TERMS; and changing "avg speed over limit" to "avg speed over limit not in 9,14 mph"**~~

~~(it had ~98% accuracy)~~

Now back to scoring ~98%

In [279]:
print(f'Percent of Stops that appeared to receive leniency == { round(100 * np.mean(df["Speed Altered"]), 3) }% \n\n')

print(f'l1_model TRAINING accuracy == {round(100 * l1_model.score(X_train, y_train), 13)}%\n')

print(f'l1_model TESTING accuracy == {round(100 * l1_model.score(X_test, y_test), 3)}% \n \n')


print(f'log_model TRAINING accuracy == {round(100 * log_model.score(X_train, y_train), 3)}%\n')

print(f'log_model TESTING accuracy == {round(100 * log_model.score(X_test, y_test), 3)}%')


# print(f'OLS_model TRAINING accuracy == {round(100 * OLS_model.score(X_train, y_train), 3)}%\n')

# print(f'OLS_model TESTING accuracy == {round(100 * OLS_model.score(X_test, y_test), 3)}%')

Percent of Stops that appeared to receive leniency == 92.914% 


l1_model TRAINING accuracy == 98.318754702784%

l1_model TESTING accuracy == 98.316% 
 

log_model TRAINING accuracy == 97.844%

log_model TESTING accuracy == 97.846%


#### Correlation Table

In [280]:
results_df.corr()

,L1 Coefficient,Normal Logit Coefficient
L1 Coefficient,1.00000,0.93199
Normal Logit Coefficient,0.93199,1.00000


### Without Intercept

Can we run this without an intercept? The intercept term is much larger than the next biggest coefficient...

In [ ]:
l1_model_no_intercept = LogisticRegression(penalty='l1', solver='liblinear', fit_intercept=False)
l1_model_no_intercept.fit(X, y)

l1_coefficients_no_intercept = l1_model_no_intercept.coef_.tolist()[0]

zipped_l1_coefs = list(zip(list(regression_df.columns), [round(x,3) for x in l1_coefficients_no_intercept]))

log_model_no_intercept = LogisticRegression(max_iter=5000, fit_intercept=False) #default max_iter==100
log_model_no_intercept.fit(X, y)

log_coefficients_no_int = log_model_no_intercept.coef_.tolist()[0]

zipped_log_coefs_no_intercept = list(zip(list(regression_df.columns), [round(x,3) for x in log_coefficients_no_int]))

no_intercept_results_df = pd.DataFrame({'Variable': [x[0] for x in zipped_l1_coefs],
                           'L1 Coefficient': [x[1] for x in zipped_l1_coefs],
                           'Normal Logit Coefficient': [x[1] for x in zipped_log_coefs_no_intercept]})

no_intercept_results_df = no_intercept_results_df.sort_values(by=['L1 Coefficient'], ascending=False)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 0):
    print(no_intercept_results_df)

In [ ]:
print(f'Percent of Stops that appeared to receive leniency == { round(100 * np.mean(df["Speed Altered"]), 3) }% \n\n')

print(f'l1_model_no_intercept TRAINING accuracy == {round(100 * l1_model_no_intercept.score(X_train, y_train), 3)}%\n')

print(f'l1_model_no_intercept TESTING accuracy == {round(100 * l1_model_no_intercept.score(X_test, y_test), 3)}% \n \n')


print(f'log_model_no_intercept TRAINING accuracy == {round(100 * log_model_no_intercept.score(X_train, y_train), 3)}%\n')

print(f'log_model_no_intercept TESTING accuracy == {round(100 * log_model_no_intercept.score(X_test, y_test), 3)}%')

#### Correlation Table

Much lower correlation than with an intercept

In [ ]:
no_intercept_results_df.corr()

# Test that each race has the same proclivity to speed (% speeds >14 mph)

An implicit assumption is that all races have the same speeding distribution - at least within the ranges of 9-14 MPH. 

\
However, this is not necessarily the case:
\

    As the following cells show, most races seem to follow their own unique speeding distribution (for speeding above 14 MPH)

In [281]:
def avg_speeds_above_14(race):
    temp_list = []
    for x in zip(df[f"{race} - (D_Race)"], df['Speed Over Posted Limit']):
        if x[0]==1 and x[1]>14:
            temp_list.append(x[1])
            
    return np.mean(temp_list)

for race in race_set:
    print(f"Average {race} speed over 14 MPH == {avg_speeds_above_14(race)}")

Average BLACK speed over 14 MPH == 21.268611232041792
Average HISPANIC speed over 14 MPH == 21.342176039119803
Average OTHER speed over 14 MPH == 20.889993972272453
Average NATIVE AMERICAN speed over 14 MPH == 21.46511627906977
Average WHITE speed over 14 MPH == 20.739335386913098
Average ASIAN speed over 14 MPH == 20.468230403800476


## Simple Z test for different means (speeding above 14 mph), with hypothesized difference between means==0

In [286]:
def z_score_above_14(race1, race2):
    x1 = np.array([x[0] for x in zip(df['Speed Over Posted Limit'], df['Race']) if x[1]==race1 and 14 < x[0] < 30])
    x2 = np.array([x[0] for x in zip(df['Speed Over Posted Limit'], df["Race"]) if x[1]==race2 and 14 < x[0] < 30])

    xbar1 = np.mean(x1)
    xbar2 = np.mean(x2)
    
    sig1 = np.std(x1)
    sig2 = np.std(x2)
    
    n1 = len(x1)
    n2 = len(x2)
    
    z = (xbar1-xbar2)/np.sqrt((sig1**2)/n1 + (sig2**2)/n2)
    
    return abs(round(z,2))

In [287]:
z_tests_above_14_table = pd.DataFrame({race1:[z_score_above_14(race1, race2) for race2 in sorted(list(race_set))] for race1 in sorted(list(race_set))}, index=sorted(list(race_set)))


z_tests_above_14_table.style.apply(lambda x: ["background: red" if v > 1.96 else "" for v in x], axis = 1)

,ASIAN,BLACK,HISPANIC,NATIVE AMERICAN,OTHER,WHITE
ASIAN,0.000000,2.660000,0.260000,1.020000,0.550000,1.050000
BLACK,2.660000,0.000000,3.190000,0.710000,3.250000,2.850000
HISPANIC,0.260000,3.190000,0.000000,0.990000,0.900000,1.050000
NATIVE AMERICAN,1.020000,0.710000,0.990000,0.000000,1.110000,0.910000
OTHER,0.550000,3.250000,0.900000,1.110000,0.000000,1.720000
WHITE,1.050000,2.850000,1.050000,0.910000,1.720000,0.000000


### Conclusion

Races probably have different speeding distributions

**We should run a test to specifically test similarity of distributions**

**Utlimately, we need to investigate whether white/asian/etc. drivers *appear* to receive more leniency because they have more stops actually at 9 mph**

# scratch